#Obtenção dos dados

##Lendo a API 'search' e colocando o resultado em uma lista

O código abaixo contém um loop que irá accessar a API várias vezes e buscar os produtos da lista 'termos' buscando 50 itens por vez até ter lido todos os items

In [122]:
import pandas as pd
import requests

termos = ['chromecast','Apple%20TV', 'Amazon%20Fire%20TV']
# cria o lista vazia para o resultado da busca
produtos = []
for x in termos:
  # controle da paginação da API
  offset=0
  while True:
    # Monta a URL da API
    urlbusca = 'https://api.mercadolibre.com/sites/MLA/search?q='+x+'&limit=50&offset=' + str(offset)
    #print(urlbusca)

    # Envia o Request para o Endpoint
    response = requests.get(urlbusca)

    if response.status_code == 200:
        # Obtem o json da resposta da API
        data = response.json()
        #print(data)

        # Colocar o json do resultado em um dataframe do pandas
        produtos.extend(data["results"])
    else:
        # Caso retornou erro printa a mensagem
        print('Error: {}'.format(response.status_code))

    # Verifica se é a última página se sim sai do loop
    if data['paging']['total'] <= offset + 50:
        break
    else:
        offset += 50



In [60]:
len(produtos)

453

##Acessando a API de Itens

Utilizando a lista de itens criada na etapa anterior, o código abaixo itera sobre a lista e para cada item da lista irá acessar a api de itens e concatenar o resultado em uma lista. Após concluir a cconstrução da lista converte a lista de dicionários em um dataframe do pandas e exporta o dataframe em CSV

In [121]:

#iterar no dataframe da busca linha a linha
items = []
for item in produtos:
  # Monta a URL de busca do item
  urlitem = 'https://api.mercadolibre.com/items/'+ item['id']
  #print(urlitem)

  # Envia o Request para o Endpoint
  response = requests.get(urlitem)
  #print(response.json())
  if response.status_code == 200:
    items.append(response.json())

  else:
    # Caso retornou erro printa a mensagem
    print('Error: {}'.format(response.status_code))
df_item = pd.DataFrame(items)
df_item.to_csv('items.csv')

#Análise

Verificando a quantidade de linhas e os dados no dataframe

In [112]:
#Verificar a quantidade de registros retornados
df_item.shape[0]

687

In [ ]:
#Verificar as primeiras linhas do dataframe
df_item.head()

Calculando o preço máximo, mínimo e média dos produtos

In [116]:
df_item['price'].describe()

count       687.000000
mean     102964.445968
std       77895.709609
min        2500.000000
25%       62622.000000
50%       84999.000000
75%      115998.500000
max      799999.000000
Name: price, dtype: float64

Verificando quantos vendedores diferentes existem no momento

In [118]:
df_item.groupby('seller_id').seller_id.nunique().count()

406

Quantidade de anúncios diferentes e anúncio mais frequente

In [120]:
df_item['title'].describe()

count                                                   687
unique                                                  549
top       Chromecast Google Tv Hd 8gb 2gb Ram 60 Fps Wif...
freq                                                     14
Name: title, dtype: object